In [2]:
import pandas as pd
import numpy as np

In [24]:
transaction = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name='Transactions')
cust_demographic = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name='CustomerDemographic')
cust_address = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name='CustomerAddress')

In [26]:
transaction.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [16]:
transaction.shape

(20000, 13)

In [27]:
cust_demographic.shape

(4000, 13)

In [28]:
cust_address.shape

(3999, 6)

In [17]:
def dataQuality(data):
    d={}
    def numeric_quality(data):
        def q1(x):
            return x.quantile(0.25)
        def q3(x):
            return x.quantile(0.75)
        def q99(x):
            return x.quantile(0.99)
        def count(x):
            return x.count()
        def miss_per(x):
            return x.isnull().sum()/len(x)
        def unique(x):
            return len(x.unique())
        qr=dict()
        #select only numeric data types
        data=data.select_dtypes(include=[np.number])
        for i in np.arange(0,len(data.columns),1):
            xi=data.agg({data.columns[i]:
                        [count,unique,miss_per,np.min,np.max,np.mean,np.median,np.std,np.var,q1,q3,q99]
                        })
            qr[data.columns[i]]=xi.reset_index(drop=True)[data.columns[i]]
            df1=pd.DataFrame(qr)
            #df1.index=xi.index
        
        df1.index=["count","unique","Miss_per","Min","Max",
                   "Mean","Median","Std","Var","Q1","Q3","q99"]
        return df1.T
    d['numeric']=numeric_quality(data)
        
    def cat_quality(data):
        def count(x):
            return x.count()
        def miss_per(x):
            return x.isnull().sum()/len(x)
        def unique(x):
            return len(x.unique())
        def freq_cat(x):
            return x.value_counts().sort_values(ascending=False).index[0]
        def freq_cat_per(x):
            return x.value_counts().sort_values(ascending=False).index[0]/len(x)
        qr=dict()
        #select only categorical data types
        data=data.select_dtypes(include=[object])
        for i in np.arange(0,len(data.columns),1):
            xi=data.agg({data.columns[i]:[count,unique,miss_per,freq_cat]})
            qr[data.columns[i]]=xi.reset_index(drop=True)[data.columns[i]]
            df2=pd.DataFrame(qr)
            #df2.index=xi.index
        df2.index=["count","Unique","Miss_per","Freq_Level"]
        return df2.T
    d['categorical']=cat_quality(data)
    return d

In [23]:
print("Data quality : Numeric variables")
(dataQuality(transaction)['numeric'])

Data quality : Numeric variables


,count,unique,Miss_per,Min,Max,Mean,Median,Std,Var,Q1,Q3,q99
transaction_id,19803.0,19803.0,0.000000,1.00,20000.00,9993.979650,9995.00,5776.499961,3.336795e+07,4988.50,15000.5,19799.98
product_id,19803.0,101.0,0.000000,0.00,100.00,45.815937,45.00,30.569841,9.345152e+02,18.00,72.0,99.00
customer_id,19803.0,3494.0,0.000000,1.00,5034.00,1739.294955,1738.00,1011.946353,1.024035e+06,858.00,2615.0,3462.98
online_order,19445.0,3.0,0.018078,0.00,1.00,0.500849,1.00,0.500012,2.500121e-01,0.00,1.0,1.00
list_price,19803.0,100.0,0.000000,12.01,2091.47,1107.997866,1163.89,582.770460,3.396214e+05,575.27,1635.3,2091.47
standard_cost,19803.0,103.0,0.000000,7.21,1759.85,556.046951,507.58,405.955660,1.648000e+05,215.14,795.1,1610.90
product_first_sold_date,19803.0,100.0,0.000000,33259.00,42710.00,38199.776549,38216.00,2875.201110,8.266781e+06,35667.00,40672.0,42710.00


In [22]:
print("Data quality : Categorical variables")
(dataQuality(transaction)['categorical'])

Data quality : Categorical variables


,count,Unique,Miss_per,Freq_Level
order_status,19803,2,0,Approved
brand,19803,6,0,Solex
product_line,19803,4,0,Standard
product_class,19803,3,0,medium
product_size,19803,3,0,medium


In [29]:
print("Data quality : Numeric variables")
(dataQuality(cust_address)['numeric'])

Data quality : Numeric variables


,count,unique,Miss_per,Min,Max,Mean,Median,Std,Var,Q1,Q3,q99
customer_id,3999.0,3999.0,0.0,1.0,4003.0,2003.987997,2004.0,1154.576912,1.333048e+06,1004.5,3003.5,3963.02
postcode,3999.0,873.0,0.0,2000.0,4883.0,2985.755939,2768.0,844.878364,7.138195e+05,2200.0,3750.0,4818.00
property_valuation,3999.0,12.0,0.0,1.0,12.0,7.514379,8.0,2.824663,7.978720e+00,6.0,10.0,12.00


In [30]:
print("Data quality : Categorical variables")
(dataQuality(cust_address)['categorical'])

Data quality : Categorical variables


,count,Unique,Miss_per,Freq_Level
address,3999,3996,0,64 Macpherson Junction
state,3999,5,0,NSW
country,3999,1,0,Australia


In [31]:
print("Data quality : Numeric variables")
(dataQuality(cust_demographic)['numeric'])

Data quality : Numeric variables


,count,unique,Miss_per,Min,Max,Mean,Median,Std,Var,Q1,Q3,q99
customer_id,4000.0,4000.0,0.00000,1.0,4000.0,2000.500000,2000.5,1154.844867,1.333667e+06,1000.75,3000.25,3960.01
past_3_years_bike_related_purchases,4000.0,100.0,0.00000,0.0,99.0,48.890000,48.0,28.715005,8.245515e+02,24.00,73.00,99.00
tenure,3913.0,23.0,0.02175,1.0,22.0,10.657041,11.0,5.660146,3.203726e+01,6.00,15.00,22.00


In [32]:
print("Data quality : Categorical variables")
(dataQuality(cust_demographic)['categorical'])

Data quality : Categorical variables


,count,Unique,Miss_per,Freq_Level
first_name,4000,3139,0,Tobe
last_name,3875,3726,0.03125,Pristnor
gender,4000,6,0,Female
job_title,3494,196,0.1265,Business Systems Development Analyst
job_industry_category,3344,10,0.164,Manufacturing
wealth_segment,4000,3,0,Mass Customer
deceased_indicator,4000,2,0,N
default,3698,91,0.0755,100
owns_car,4000,2,0,Yes


In [35]:
cust_demographic['wealth_segment'].unique()

array(['Mass Customer', 'Affluent Customer', 'High Net Worth'],
      dtype=object)